In [1]:
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import Team
from basketball_reference_web_scraper.data import OutputType
import pandas as pd
import datetime as datetime
from datetime import datetime, timedelta
from dateutil.parser import parse
from pulp import *
import numpy as np
from sklearn import linear_model

In [ ]:
for m in range(1, 3):
    if m == 2:
        for d in range(1, 29):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            client.player_box_scores(day=d, month=m, year=2020, output_type=OutputType.CSV, output_file_path=file_name)
    else:
        for d in range(1, 32):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            client.player_box_scores(day=d, month=m, year=2020, output_type=OutputType.CSV, output_file_path=file_name)

In [3]:
client.player_box_scores(day=1, month=1, year=2020)

[{'slug': 'jamesle01',
  'name': 'LeBron James',
  'team': <Team.LOS_ANGELES_LAKERS: 'LOS ANGELES LAKERS'>,
  'location': <Location.HOME: 'HOME'>,
  'opponent': <Team.PHOENIX_SUNS: 'PHOENIX SUNS'>,
  'outcome': <Outcome.WIN: 'WIN'>,
  'seconds_played': 2284,
  'made_field_goals': 11,
  'attempted_field_goals': 21,
  'made_three_point_field_goals': 1,
  'attempted_three_point_field_goals': 4,
  'made_free_throws': 8,
  'attempted_free_throws': 12,
  'offensive_rebounds': 2,
  'defensive_rebounds': 11,
  'assists': 12,
  'steals': 2,
  'blocks': 1,
  'turnovers': 5,
  'personal_fouls': 0,
  'game_score': 29.9},
 {'slug': 'antetgi01',
  'name': 'Giannis Antetokounmpo',
  'team': <Team.MILWAUKEE_BUCKS: 'MILWAUKEE BUCKS'>,
  'location': <Location.HOME: 'HOME'>,
  'opponent': <Team.MINNESOTA_TIMBERWOLVES: 'MINNESOTA TIMBERWOLVES'>,
  'outcome': <Outcome.WIN: 'WIN'>,
  'seconds_played': 1889,
  'made_field_goals': 13,
  'attempted_field_goals': 22,
  'made_three_point_field_goals': 3,
  'atte

In [2]:
all_tables = []
for m in range(1, 3):
    if m == 2:
        for d in range(1, 29):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            table  = pd.read_csv(file_name)
            date = datetime(2020, m, d)
            dates = [date] * len(table)
            table["Date"] = dates
            all_tables.append(table)

    else:
        for d in range(1, 32):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            table  = pd.read_csv(file_name)
            date = datetime(2020, m, d)
            dates = [date] * len(table)
            table["Date"] = dates
            all_tables.append(table)

In [3]:
full_df = all_tables[0]
for i in range(1, len(all_tables)):
    current_table = all_tables[i]
    full_df = full_df.append(current_table)
full_df.to_csv("./OutputCSVs/all_games.csv")

In [4]:
df = pd.read_csv("./OutputCSVs/teams_stats.csv")
df["Team"] = df["Team"].str.upper()
all_games = pd.read_csv("./OutputCSVs/all_games.csv")
x = all_games[["team", "opponent"]].loc[0]["team"]
df[df["Team"] == x]["TOV%"]

team_def = []
team_pace = []
team_tov = []
opp_def = []
opp_pace = []
opp_tov = []
all_games_teams = all_games[["team", "opponent"]]

for i in range(len(all_games_teams)):
    game = all_games_teams.loc[i]
    team = game["team"]
    opponent = game["opponent"]
    team_def.append(df[df["Team"] == team]["DRtg"].iloc[0])
    team_pace.append(df[df["Team"] == team]["Pace"].iloc[0])
    team_tov.append(df[df["Team"] == team]["TOV%"].iloc[0])
    opp_def.append(df[df["Team"] == opponent]["DRtg"].iloc[0])
    opp_pace.append(df[df["Team"] == opponent]["Pace"].iloc[0])
    opp_tov.append(df[df["Team"] == opponent]["TOV%"].iloc[0])
    
all_games["Team Defensive Rating"] = team_def
all_games["Team Pace"] = team_pace
all_games["Team Turnover %"] = team_tov
all_games["Opponent Defensive Rating"] = opp_def
all_games["Opponent Pace"] = opp_pace
all_games["Opponent Turnover %"] = opp_tov

all_games.to_csv("./OutputCSVs/all_games_new.csv")
df = pd.read_csv("./OutputCSVs/all_games_new.csv")
attempted_2s = df["attempted_field_goals"] - df["attempted_three_point_field_goals"]
made_2s = df["made_field_goals"] - df["made_three_point_field_goals"]
rebounds = df["offensive_rebounds"] + df["defensive_rebounds"]
at_home = df["location"] == "HOME"
df["attempted_two_point_field_goals"] = attempted_2s
df["made_two_point_field_goals"] = made_2s
df["total_rebounds"] = rebounds
df["at_home"] = at_home
df.to_csv("./OutputCSVs/all_games_updated.csv")
all_games_actual = pd.read_csv("./OutputCSVs/all_games_updated.csv")
all_games_actual =  all_games_actual.iloc[:, 3:].reset_index()

In [5]:
def get_stats(player, date, number_rows):
    converted_datetime = datetime.strptime(date, '%Y-%m-%d')
    player_rows = all_games_actual.loc[all_games_actual['name'] == player]
    #print(player_rows)
    selected_rows = []
    if (len(player_rows)) < number_rows:
        for i in range(len(player_rows)):
            selected_rows.append(player_rows.iloc[i])
        return pd.DataFrame(selected_rows)
    index = 0
    for i in range(len(player_rows)):
        curr_date = player_rows.iloc[i]['Date']
        if datetime.strptime(curr_date, '%Y-%m-%d') >= converted_datetime:
            index = i - 1
            break
    if index != 0:
        if index + 1 - number_rows < 0:
            for i in range(index + 1):
                selected_rows.append(player_rows.iloc[i])
        else:
            for i in range(index + 1 - number_rows, index + 1):
                selected_rows.append(player_rows.iloc[i])
#     elif converted_datetime >= players_rows[len(player_rows) - 1]:
    else:
        for i in range(len(player_rows) - number_rows, len(player_rows)):
            selected_rows.append(player_rows.iloc[i])
    return pd.DataFrame(selected_rows)

In [6]:
def get_points(row_data):
    three_pt_fgs = row_data['made_three_point_field_goals']
    two_pt_fgs = row_data['made_two_point_field_goals']
    made_fts = row_data['made_free_throws']
    total_rebounds = row_data['rebounds']
    assists = row_data['assists']
    blocks = row_data['blocks']
    steals = row_data['steals']
    turnovers = row_data['turnovers']
    FD_points = three_pt_fgs * 3 + two_pt_fgs * 2 + made_fts + total_rebounds * 1.2 + assists * 1.5 + blocks * 3 + steals * 3 - turnovers
    FD_dollars = FD_points * 200
    return (FD_points, FD_dollars)

In [7]:
def weight_function(statistic, weight):
    s = 0
    if type(statistic) == np.ndarray:
        for i in range(len(statistic)):
            s += statistic[len(statistic) - i - 1] * (weight ** i)
    else:
        for i in range(len(statistic)):
            s += statistic.iloc[len(statistic) - i - 1,] * (weight ** i)
    return s

In [8]:
def time_weighted_average(rows, statistic, weight):
    if rows.empty:
        return 0
    stat = rows[statistic]
    this_num = 1 / weight_function(np.ones(len(stat)), weight)
    return this_num * weight_function(stat, weight)
player_box_scores = pd.read_csv('./OutputCSVs/all_games_updated.csv')

input_statistics = ["name", "team", "date", "location", "opponent", "made_field_goals", "made_two_point_field_goals", "attempted_two_point_field_goals", "attempted_field_goals", "made_three_point_field_goals", "attempted_three_point_field_goals", "attempted_free_throws", "made_free_throws", "offensive_rebounds", "defensive_rebounds", "assists", "blocks", "turnovers", "steals", "seconds_played"]

In [25]:
def statline_predictor(player_box_scores, input_statistics, sample_size = 5, weight = .8):
    player_box_scores = player_box_scores[~player_box_scores.index.duplicated()]
    player_box_scores.reindex(range(len(player_box_scores)), axis = "index")
    predicted_statlines = pd.DataFrame(index = player_box_scores.index, columns = input_statistics).fillna(0).T
    for box_index in player_box_scores.index:
        box_score = player_box_scores.loc[box_index]
        player_name = box_score["name"]
        game_date = str(box_score["Date"])[:10]
        if (box_index % 50 == 0):
            print(player_name)
            print(box_index)
            print(game_date)
        last_n_rows = get_stats(player_name, game_date, sample_size)
        weighted_stats = [player_name, box_score["team"], game_date, box_score["location"], box_score["opponent"]]
        for stat in input_statistics[5:]:
            weighted_stats.append(round(time_weighted_average(last_n_rows, stat, weight), 2))
        predicted_statlines[box_index] = weighted_stats
    return predicted_statlines.T

In [10]:
predictions_samplesize5 = statline_predictor(all_games_actual, input_statistics, sample_size = 5)
predictions_samplesize5.to_csv("./OutputCSVs/predicted_statlines_trailing_5_games.csv")
predictions_samplesize10 = statline_predictor(all_games_actual, input_statistics, sample_size = 10)
predictions_samplesize10.to_csv("./OutputCSVs/predicted_statlines_trailing_10_games.csv")

LeBron James
0
2020-01-01
Giannis Antetokounmpo
1
2020-01-01
D.J. Augustin
2
2020-01-01
Kelly Oubre
3
2020-01-01
Mitchell Robinson
4
2020-01-01
Anthony Davis
5
2020-01-01
Hassan Whiteside
6
2020-01-01
Devin Booker
7
2020-01-01
Julius Randle
8
2020-01-01
Bradley Beal
9
2020-01-01
Carmelo Anthony
10
2020-01-01
Avery Bradley
11
2020-01-01
Shabazz Napier
12
2020-01-01
Markelle Fultz
13
2020-01-01
Frank Ntilikina
14
2020-01-01
Deandre Ayton
15
2020-01-01
De'Aaron Fox
16
2020-01-02
Jeremy Lamb
17
2020-01-02
Luka Dončić
18
2020-01-02
Shai Gilgeous-Alexander
19
2020-01-02
Domantas Sabonis
20
2020-01-02
Robert Covington
21
2020-01-02
LaMarcus Aldridge
22
2020-01-02
Terry Rozier
23
2020-01-02
Seth Curry
24
2020-01-02
Michael Porter
25
2020-01-02
Jamal Murray
26
2020-01-02
Shabazz Napier
27
2020-01-02
Buddy Hield
28
2020-01-02
DeMar DeRozan
29
2020-01-02
Maxi Kleber
30
2020-01-02
T.J. Warren
31
2020-01-02
Kawhi Leonard
32
2020-01-02
Jonas Valančiūnas
33
2020-01-02
Wendell Carter
34
2020-01-02
Jae

Romeo Langford
294
2020-01-03
Raul Neto
295
2020-01-03
Cam Reddish
296
2020-01-03
Isaiah Thomas
297
2020-01-03
P.J. Tucker
298
2020-01-03
Wayne Ellington
299
2020-01-03
Tremont Waters
300
2020-01-03
Cameron Johnson
301
2020-01-03
Bruno Fernando
302
2020-01-03
Anžejs Pasečņiks
303
2020-01-03
Anfernee Simons
304
2020-01-03
Anthony Tolliver
305
2020-01-03
Kenrich Williams
306
2020-01-03
Jaylen Hoard
307
2020-01-03
Ben McLemore
308
2020-01-03
DeAndre' Bembry
309
2020-01-03
Kentavious Caldwell-Pope
310
2020-01-03
Nassir Little
311
2020-01-03
Kelly Olynyk
312
2020-01-03
Dario Šarić
313
2020-01-03
Isaiah Hartenstein
314
2020-01-03
Cheick Diallo
315
2020-01-03
Admiral Schofield
316
2020-01-03
Mike Scott
317
2020-01-03
Tyler Herro
318
2020-01-03
Jamal Murray
319
2020-01-04
Trae Young
320
2020-01-04
Jae Crowder
321
2020-01-04
Ish Smith
322
2020-01-04
Luka Dončić
323
2020-01-04
Zach LaVine
324
2020-01-04
Giannis Antetokounmpo
325
2020-01-04
Terry Rozier
326
2020-01-04
Troy Brown
327
2020-01-04
De

Kelly Oubre
588
2020-01-05
Alex Caruso
589
2020-01-05
Meyers Leonard
590
2020-01-05
Sviatoslav Mykhailiuk
591
2020-01-05
Dwight Howard
592
2020-01-05
JaMychal Green
593
2020-01-05
Reggie Bullock
594
2020-01-05
JaVale McGee
595
2020-01-05
Mario Hezonja
596
2020-01-05
Collin Sexton
597
2020-01-05
Kendrick Nunn
598
2020-01-05
John Henson
599
2020-01-05
Andre Drummond
600
2020-01-05
Christian Wood
601
2020-01-05
Julius Randle
602
2020-01-05
Dillon Brooks
603
2020-01-05
Brandon Clarke
604
2020-01-05
Tyler Herro
605
2020-01-05
Andrew Wiggins
606
2020-01-05
Solomon Hill
607
2020-01-05
Taj Gibson
608
2020-01-05
Patrick Beverley
609
2020-01-05
Ricky Rubio
610
2020-01-05
Naz Reid
611
2020-01-05
Mitchell Robinson
612
2020-01-05
Ja Morant
613
2020-01-05
Jeff Teague
614
2020-01-05
Kadeem Allen
615
2020-01-05
De'Anthony Melton
616
2020-01-05
Darius Garland
617
2020-01-05
Tyus Jones
618
2020-01-05
Aron Baynes
619
2020-01-05
Kyle Anderson
620
2020-01-05
Frank Ntilikina
621
2020-01-05
Bruce Brown
622
2

Nemanja Bjelica
882
2020-01-07
Elfrid Payton
883
2020-01-07
Oshae Brissett
884
2020-01-07
Danny Green
885
2020-01-07
Tristan Thompson
886
2020-01-07
Caris LeVert
887
2020-01-07
Collin Sexton
888
2020-01-07
Brandon Knight
889
2020-01-07
Chris Boucher
890
2020-01-07
Serge Ibaka
891
2020-01-07
Kelly Oubre
892
2020-01-07
Kentavious Caldwell-Pope
893
2020-01-07
Darius Garland
894
2020-01-07
Mitchell Robinson
895
2020-01-07
Cedi Osman
896
2020-01-07
Bruce Brown
897
2020-01-07
Kyle Kuzma
898
2020-01-07
Sviatoslav Mykhailiuk
899
2020-01-07
Ricky Rubio
900
2020-01-07
Solomon Hill
901
2020-01-07
Buddy Hield
902
2020-01-07
Avery Bradley
903
2020-01-07
Julius Randle
904
2020-01-07
JaVale McGee
905
2020-01-07
Naz Reid
906
2020-01-07
Rajon Rondo
907
2020-01-07
Harry Giles
908
2020-01-07
Cory Joseph
909
2020-01-07
Dewayne Dedmon
910
2020-01-07
Garrett Temple
911
2020-01-07
Andrew Wiggins
912
2020-01-07
Ante Žižić
913
2020-01-07
DeAndre Jordan
914
2020-01-07
RJ Barrett
915
2020-01-07
Anfernee Simons
9

Rudy Gay
1172
2020-01-08
DeAndre' Bembry
1173
2020-01-08
Oshae Brissett
1174
2020-01-08
Tristan Thompson
1175
2020-01-09
Danilo Gallinari
1176
2020-01-09
Andre Drummond
1177
2020-01-09
Josh Richardson
1178
2020-01-09
Bruce Brown
1179
2020-01-09
Andrew Wiggins
1180
2020-01-09
Al Horford
1181
2020-01-09
Marcus Smart
1182
2020-01-09
Chris Paul
1183
2020-01-09
Damian Lillard
1184
2020-01-09
Kemba Walker
1185
2020-01-09
Shai Gilgeous-Alexander
1186
2020-01-09
Russell Westbrook
1187
2020-01-09
Steven Adams
1188
2020-01-09
Darius Garland
1189
2020-01-09
Derrick Rose
1190
2020-01-09
Collin Sexton
1191
2020-01-09
Ben Simmons
1192
2020-01-09
Gorgui Dieng
1193
2020-01-09
Hassan Whiteside
1194
2020-01-09
Robert Covington
1195
2020-01-09
Gary Trent
1196
2020-01-09
Tobias Harris
1197
2020-01-09
Jayson Tatum
1198
2020-01-09
Daniel Theis
1199
2020-01-09
Jeff Teague
1200
2020-01-09
Kevin Love
1201
2020-01-09
Josh Okogie
1202
2020-01-09
Chris Clemons
1203
2020-01-09
James Harden
1204
2020-01-09
Clint Ca

...

Rayjon Tucker
6860
2020-02-21
JaVale McGee
6861
2020-02-21
Grant Williams
6862
2020-02-21
Jared Dudley
6863
2020-02-21
Stanley Johnson
6864
2020-02-21
Malcolm Miller
6865
2020-02-21
Juwan Morgan
6866
2020-02-21
Justin Wright-Foreman
6867
2020-02-21
Moritz Wagner
6868
2020-02-21
Matthew Dellavedova
6869
2020-02-21
Victor Oladipo
6870
2020-02-21
James Ennis
6871
2020-02-21
Cheick Diallo
6872
2020-02-21
Frank Jackson
6873
2020-02-21
Wenyen Gabriel
6874
2020-02-21
Enes Kanter
6875
2020-02-21
Ty Jerome
6876
2020-02-21
Michael Kidd-Gilchrist
6877
2020-02-21
D.J. Augustin
6878
2020-02-21
Tony Bradley
6879
2020-02-21
Dillon Brooks
6880
2020-02-21
Gary Clark
6881
2020-02-21
Terrance Ferguson
6882
2020-02-21
Romeo Langford
6883
2020-02-21
Paul Millsap
6884
2020-02-21
Kevin Porter
6885
2020-02-21
Elie Okobo
6886
2020-02-21
De'Anthony Melton
6887
2020-02-21
Allen Crabbe
6888
2020-02-21
John Collins
6889
2020-02-22
James Harden
6890
2020-02-22
Giannis Antetokounmpo
6891
2020-02-22
Kendrick Nunn
689

OG Anunoby
7137
2020-02-23
Sekou Doumbouya
7138
2020-02-23
Derrick Rose
7139
2020-02-23
Daniel Gafford
7140
2020-02-23
Thon Maker
7141
2020-02-23
Josh Hart
7142
2020-02-23
Stanley Johnson
7143
2020-02-23
Alex Caruso
7144
2020-02-23
Jeremy Lamb
7145
2020-02-23
Fred VanVleet
7146
2020-02-23
Sviatoslav Mykhailiuk
7147
2020-02-23
Patty Mills
7148
2020-02-23
Lonnie Walker
7149
2020-02-23
Deonte Burton
7150
2020-02-23
LaMarcus Aldridge
7151
2020-02-23
Avery Bradley
7152
2020-02-23
J.J. Redick
7153
2020-02-23
Ian Mahinmi
7154
2020-02-23
Grant Williams
7155
2020-02-23
Malcolm Miller
7156
2020-02-23
Semi Ojeleye
7157
2020-02-23
Tony Snell
7158
2020-02-23
Derrick White
7159
2020-02-23
Kevon Looney
7160
2020-02-23
Jerome Robinson
7161
2020-02-23
Noah Vonleh
7162
2020-02-23
Paul Watson
7163
2020-02-23
T.J. Leaf
7164
2020-02-23
Mario Hezonja
7165
2020-02-23
Romeo Langford
7166
2020-02-23
Isaiah Roby
7167
2020-02-23
Torrey Craig
7168
2020-02-23
E'Twaun Moore
7169
2020-02-23
Jordan McRae
7170
2020-02

Lonzo Ball
7416
2020-02-25
George Hill
7417
2020-02-25
Kentavious Caldwell-Pope
7418
2020-02-25
Nicolò Melli
7419
2020-02-25
JaKarr Sampson
7420
2020-02-25
Daniel Theis
7421
2020-02-25
Matt Thomas
7422
2020-02-25
Marvin Williams
7423
2020-02-25
Thaddeus Young
7424
2020-02-25
Chris Boucher
7425
2020-02-25
Eric Paschall
7426
2020-02-25
Rondae Hollis-Jefferson
7427
2020-02-25
Abdel Nader
7428
2020-02-25
Marcus Smart
7429
2020-02-25
Aaron Holiday
7430
2020-02-25
Donte DiVincenzo
7431
2020-02-25
Gary Harris
7432
2020-02-25
Andrew Wiggins
7433
2020-02-25
Derrick Favors
7434
2020-02-25
Gordon Hayward
7435
2020-02-25
Mason Plumlee
7436
2020-02-25
Caleb Swanigan
7437
2020-02-25
Terence Davis
7438
2020-02-25
Langston Galloway
7439
2020-02-25
Caleb Martin
7440
2020-02-25
Carmelo Anthony
7441
2020-02-25
Cody Zeller
7442
2020-02-25
Daniel Gafford
7443
2020-02-25
Doug McDermott
7444
2020-02-25
Willy Hernangómez
7445
2020-02-25
Cody Martin
7446
2020-02-25
Kent Bazemore
7447
2020-02-25
Will Barton
744

Steven Adams
7703
2020-02-27
Maurice Harkless
7704
2020-02-27
Harrison Barnes
7705
2020-02-27
Gary Trent
7706
2020-02-27
Malcolm Brogdon
7707
2020-02-27
Nerlens Noel
7708
2020-02-27
Rajon Rondo
7709
2020-02-27
Shai Gilgeous-Alexander
7710
2020-02-27
Mitchell Robinson
7711
2020-02-27
Cory Joseph
7712
2020-02-27
Jordan Poole
7713
2020-02-27
Avery Bradley
7714
2020-02-27
Myles Turner
7715
2020-02-27
Alex Len
7716
2020-02-27
Victor Oladipo
7717
2020-02-27
Carmelo Anthony
7718
2020-02-27
T.J. Warren
7719
2020-02-27
Josh Richardson
7720
2020-02-27
RJ Barrett
7721
2020-02-27
Justin Holiday
7722
2020-02-27
Dennis Schröder
7723
2020-02-27
JaVale McGee
7724
2020-02-27
Buddy Hield
7725
2020-02-27
Kentavious Caldwell-Pope
7726
2020-02-27
T.J. McConnell
7727
2020-02-27
Furkan Korkmaz
7728
2020-02-27
Kyle Kuzma
7729
2020-02-27
Markieff Morris
7730
2020-02-27
Damion Lee
7731
2020-02-27
Aaron Holiday
7732
2020-02-27
Caleb Swanigan
7733
2020-02-27
Kyle O'Quinn
7734
2020-02-27
Marquese Chriss
7735
2020-

Patrick McCaw
7982
2020-02-28
Chris Chiozza
7983
2020-02-28


In [11]:
def rebound_predictor_5(weighted_stats):
    if "offensive_rebounds_y" in weighted_stats.index:
        return 3.495 + .9489 * weighted_stats["offensive_rebounds_y"] + .2874 * weighted_stats["defensive_rebounds_y"]
    else:
        return 3.495 + .9489 * weighted_stats["offensive_rebounds"] + .2874 * weighted_stats["defensive_rebounds"]
def freethrow_predictor_5(weighted_stats):
    if "attempted_free_throws_y" in weighted_stats.index:
        return 1.373 + .6217 * weighted_stats["attempted_free_throws_y"] + -.0373 * weighted_stats["attempted_field_goals_y"] -.3289 * (weighted_stats["location_y"] == "HOME")
    else:
        return 1.373 + .6217 * weighted_stats["attempted_free_throws"] + -.0373 * weighted_stats["attempted_field_goals"] -.3289 * (weighted_stats["location"] == "HOME")
def twopt_predictor_5(weighted_stats):
    if "attempted_field_goals_y" in weighted_stats.index:
        return max(0, -5.395 + .6064 * weighted_stats["attempted_two_point_field_goals_y"] - .2542 * weighted_stats["attempted_field_goals_y"] + .071 * weighted_stats["Opponent Defensive Rating"])
    else:
        return max(0, -5.395 + .6064 * weighted_stats["attempted_two_point_field_goals"] - .2542 * weighted_stats["attempted_field_goals"] + .071 * weighted_stats["Opponent Defensive Rating"])
def threept_predictor_5(weighted_stats):
    if "attempted_three_point_field_goals_y" in weighted_stats.index:
        return .766 + .214 * weighted_stats["attempted_three_point_field_goals_y"]
    else:
        return .766 + .214 * weighted_stats["attempted_three_point_field_goals"]
def assist_predictor_5(weighted_stats):
    if "assists_y" in weighted_stats.index:
        return 2.06 + .532 * weighted_stats["assists_y"] - .064 * weighted_stats["blocks_y"]
    else:
        return 2.06 + .532 * weighted_stats["assists"] - .064 * weighted_stats["blocks"]
def block_predictor_5(weighted_stats):
    if "offensive_rebounds_y" in weighted_stats.index:
        return .329 + .182 * weighted_stats["offensive_rebounds_y"] - .038 * weighted_stats["defensive_rebounds_y"] + .424 * weighted_stats["blocks_y"]
    else:
        return .329 + .182 * weighted_stats["offensive_rebounds"] - .038 * weighted_stats["defensive_rebounds"] + .424 * weighted_stats["blocks"]
def steal_predictor_5(weighted_stats):
    if "steals_y" in weighted_stats.index:
        return max(0, -.936 + .339 * weighted_stats["steals_y"] + .129 * weighted_stats["Opponent Turnover %"])
    else:
        return max(0, -.936 + .339 * weighted_stats["steals"] + .129 * weighted_stats["Opponent Turnover %"])
def turnover_predictor_5(weighted_stats):
    if "turnovers_y" in weighted_stats.index:
        return max(0, -2.589 + .18 * weighted_stats["turnovers_y"] + .121 * weighted_stats["assists_y"] + .032 * weighted_stats["Opponent Defensive Rating"])
    else:
        return max(0, -2.589 + .18 * weighted_stats["turnovers"] + .121 * weighted_stats["assists"] + .032 * weighted_stats["Opponent Defensive Rating"])
def minutes_predictor(weighted_stats):
    if "seconds_played_y" in weighted_stats.index:
        return np.mean(weighted_stats["seconds_played_y"])/60
    else:
        return np.mean(weighted_stats["seconds_played"])/60
def recent_average(weighted_stats):
    if "made_three_point_field_goals_y" in weighted_stats.index:
        return 3*np.mean(weighted_stats["made_three_point_field_goals_y"]) + 2*np.mean(weighted_stats["made_two_point_field_goals_y"]) + np.mean(weighted_stats["made_free_throws_y"]) + 1.2*(np.mean(weighted_stats["offensive_rebounds_y"]) + np.mean(weighted_stats["defensive_rebounds_y"])) + 1.5*np.mean(weighted_stats["assists_y"]) + 3*np.mean(weighted_stats["blocks_y"]) + 3*np.mean(weighted_stats["steals_y"]) - np.mean(weighted_stats["turnovers_y"])
    else:
        return 3*np.mean(weighted_stats["made_three_point_field_goals"]) + 2*np.mean(weighted_stats["made_two_point_field_goals"]) + np.mean(weighted_stats["made_free_throws"]) + 1.2*(np.mean(weighted_stats["offensive_rebounds"]) + np.mean(weighted_stats["defensive_rebounds"])) + 1.5*np.mean(weighted_stats["assists"]) + 3*np.mean(weighted_stats["blocks"]) + 3*np.mean(weighted_stats["steals"]) - np.mean(weighted_stats["turnovers"])

In [12]:
all_games_predicted_5 = pd.read_csv("./OutputCSVs/predicted_statlines_trailing_5_games.csv")
all_games_predicted_5 = all_games_predicted_5.rename(columns={"Unnamed: 0": "indexr"})
all_games_merged_5 = all_games_actual.merge(all_games_predicted_5, left_on="index", right_on="indexr")
all_games_predicted_10 = pd.read_csv("./OutputCSVs/predicted_statlines_trailing_10_games.csv")
all_games_predicted_10 = all_games_predicted_10.rename(columns={"Unnamed: 0": "indexr"})
all_games_merged_10 = all_games_actual.merge(all_games_predicted_10, left_on="index", right_on="indexr")
all_games_merged_5.to_csv("./OutputCSVs/true_and_predicted_statlines_5.csv")
all_games_merged_10.to_csv("./OutputCSVs/true_and_predicted_statlines_10.csv")

In [13]:
weighted_preds = pd.read_csv("./OutputCSVs/true_and_predicted_statlines_5.csv")

In [14]:
output_statistics = ["name", "team", "date", "location", "opponent", "minutes", "made_two_point_field_goals", "made_three_point_field_goals", "made_free_throws", "rebounds", "assists", "blocks", "steals", "turnovers", "recent_average"]

def statline_output(weighted_predicted_lines):
    output_statlines = pd.DataFrame(index = weighted_predicted_lines.index, columns = output_statistics).fillna(0).T
    for box_index in weighted_predicted_lines.index:
        weighted_line = weighted_predicted_lines.loc[box_index]
        if "name_x" in weighted_line.index:
            output_stats = [weighted_line["name_x"], weighted_line["team_x"], weighted_line["Date"], weighted_line["location_x"], weighted_line["opponent_x"]]
        else:
            output_stats = [weighted_line["name"], weighted_line["team"], weighted_line["date"], weighted_line["location"], weighted_line["opponent"]]
        output_stats.append(minutes_predictor(weighted_line))
        output_stats.append(twopt_predictor_5(weighted_line))
        output_stats.append(threept_predictor_5(weighted_line))
        output_stats.append(freethrow_predictor_5(weighted_line))
        output_stats.append(rebound_predictor_5(weighted_line))
        output_stats.append(assist_predictor_5(weighted_line))
        output_stats.append(block_predictor_5(weighted_line))
        output_stats.append(steal_predictor_5(weighted_line))
        output_stats.append(block_predictor_5(weighted_line))
        output_stats.append(recent_average(weighted_line))
        output_statlines[box_index] = output_stats
    return output_statlines.T
output_picks = statline_output(weighted_preds)
full_test_output = output_picks[output_picks["date"] > "2020-01-01"]
full_test_output.to_csv("./OutputCSVs/predicted_statlines_jan_feb_2020.csv")

In [15]:
jan_feb_data = pd.read_csv('./OutputCSVs/predicted_statlines_jan_feb_2020.csv')
FD_points_array = []
FD_dollars_array = []
sum_points = 0
counter = 0
for index, row in jan_feb_data.iterrows():
    tuple_values = get_points(row)
    FD_points_array.append(tuple_values[0])
    FD_dollars_array.append(tuple_values[1])
    sum_points += tuple_values[0]
    counter += 1
jan_feb_data['FanDuel_Points'] = FD_points_array
jan_feb_data['FanDuel_Dollars'] = FD_dollars_array
jan_feb_data.to_csv("./OutputCSVs/Jan_Feb_statline_with_FanDuel_dollars.csv")

In [16]:
def generate_optimal_lineup(players, positions, values, costs, budget):
    num_variables = len(players)
    
    lp = LpProblem("My LP Problem", pulp.LpMaximize)
    
    d = {}
    for i in range(0, num_variables):
        d[players[i]] = LpVariable(players[i], cat="Binary")
    
    objective = sum(np.array(values) * np.array(list(d.values())))
    lp += objective
    
    pg_constraint = 0
    sg_constraint = 0
    sf_constraint = 0
    pf_constraint = 0
    c_constraint = 0
    for i in range(0, len(positions)):
        if positions[i] == "PG":
            pg_constraint += d[players[i]]
        elif positions[i] == "SG":
            sg_constraint += d[players[i]]
        elif positions[i] == "SF":
            sf_constraint += d[players[i]]
        elif positions[i] == "PF":
            pf_constraint += d[players[i]]
        else:
            c_constraint += d[players[i]]
    lp += pg_constraint == 2
    lp += sg_constraint == 2
    lp += sf_constraint == 2
    lp += pf_constraint == 2
    lp += c_constraint == 1
    
    cost = sum(np.array(costs) * np.array(list(d.values())))
    lp += cost <= budget
    
    lp.solve()
    
    lineup = [variable.name for variable in lp.variables() if variable.varValue == 1]
    print(lineup)
    #for variable in lp.variables():
    #    print ("{} = {}".format(variable.name, variable.varValue))
    #print(pulp.value(lp.objective))

In [17]:
def box_scores_for_range_of_days(start_date, end_date):
    all_tables = []
    start_month = start_date.month
    end_month = end_date.month
    for m in range(start_month, end_month + 1):
        if m == start_month:
            start_day = start_date.day
        else:
            start_day = 1
        if m == end_month:
            end_day = end_date.day
        else:
            if m == 2:
                end_day = 29
            elif m in [9, 4, 6, 11]:
                end_day = 31
            else:
                end_day = 32
        
        for d in range(start_day, end_day + 1):
            file_name = "./AllCSVs/{0}_{1}_2020_box_scores.csv".format(m, d)
            if pd.read_csv(file_name).empty:
                client.player_box_scores(day=d, month=m, year=2020, output_type=OutputType.CSV, output_file_path=file_name)
            table = pd.read_csv(file_name)
            date = datetime(2020, m, d)
            dates = [date] * len(table)
            table["Date"] = dates
            all_tables.append(table)

    full_df = all_tables[0]
    for i in range(1, len(all_tables)):
        current_table = all_tables[i]
        full_df = full_df.append(current_table)
    
    full_df.index = range(full_df.shape[0])
    df = pd.read_csv("./OutputCSVs/teams_stats.csv")
    df["Team"] = df["Team"].str.upper()

    team_def = []
    team_pace = []
    team_tov = []
    opp_def = []
    opp_pace = []
    opp_tov = []
    all_games_teams = full_df[["team", "opponent"]]

    for i in range(len(all_games_teams)):
        game = all_games_teams.loc[i]
        team = game["team"].upper()
        opponent = game["opponent"].upper()
        team_def.append(df[df["Team"] == team]["DRtg"].iloc[0])
        team_pace.append(df[df["Team"] == team]["Pace"].iloc[0])
        team_tov.append(df[df["Team"] == team]["TOV%"].iloc[0])
        opp_def.append(df[df["Team"] == opponent]["DRtg"].iloc[0])
        opp_pace.append(df[df["Team"] == opponent]["Pace"].iloc[0])
        opp_tov.append(df[df["Team"] == opponent]["TOV%"].iloc[0])
    
    full_df["Team Defensive Rating"] = team_def
    full_df["Team Pace"] = team_pace
    full_df["Team Turnover %"] = team_tov
    full_df["Opponent Defensive Rating"] = opp_def
    full_df["Opponent Pace"] = opp_pace
    full_df["Opponent Turnover %"] = opp_tov
    return full_df

In [21]:
def make_predictions(start_date, end_date, output = True):
    input_statistics = ["name", "team", "date", "location", "opponent", "made_field_goals", "made_two_point_field_goals", "attempted_two_point_field_goals", "attempted_field_goals", "made_three_point_field_goals", "attempted_three_point_field_goals", "attempted_free_throws", "made_free_throws", "offensive_rebounds", "defensive_rebounds", "assists", "blocks", "turnovers", "steals", "seconds_played", "Opponent Defensive Rating", "Opponent Turnover %"]
    full_df = box_scores_for_range_of_days(start_date, end_date)
    predicted_statlines = statline_output(statline_predictor(full_df, input_statistics))
    if output:
        output_filename = './AllCSVs/' + str(start_date.month) + '_' + str(start_date.day) + '_' + str(end_date.month) + '_' + str(end_date.day) + '_2020_predicted_box_scores.csv'
        predicted_statlines.to_csv(output_filename)
    return predicted_statlines

make_predictions(datetime(2020, 2, 1), datetime(2020, 2, 11), output = True)

Damian Lillard
0
2020-02-01
Bradley Beal
1
2020-02-01
Karl-Anthony Towns
2
2020-02-01
Dorian Finney-Smith
3
2020-02-01
Anthony Davis
4
2020-02-01
Kawhi Leonard
5
2020-02-01
Jalen Brunson
6
2020-02-01
Jaylen Brown
7
2020-02-01
DeMar DeRozan
8
2020-02-01
Cody Zeller
9
2020-02-01
Glenn Robinson
10
2020-02-01
Hassan Whiteside
11
2020-02-01
Collin Sexton
12
2020-02-01
Maxi Kleber
13
2020-02-01
Draymond Green
14
2020-02-01
Nikola Vučević
15
2020-02-01
Jayson Tatum
16
2020-02-01
Jarrett Allen
17
2020-02-01
John Collins
18
2020-02-01
Ben Simmons
19
2020-02-01
Tyler Herro
20
2020-02-01
Jakob Poeltl
21
2020-02-01
Aaron Gordon
22
2020-02-01
Bojan Bogdanović
23
2020-02-01
Miles Bridges
24
2020-02-01
Marcus Morris
25
2020-02-01
D'Angelo Russell
26
2020-02-01
Jimmy Butler
27
2020-02-01
Domantas Sabonis
28
2020-02-01
Paul George
29
2020-02-01
Buddy Hield
30
2020-02-01
Donovan Mitchell
31
2020-02-01
Harry Giles
32
2020-02-01
LeBron James
33
2020-02-01
Damion Lee
34
2020-02-01
Joe Harris
35
2020-02-01


Tyler Johnson
295
2020-02-02
P.J. Tucker
296
2020-02-02
Wesley Matthews
297
2020-02-02
Gary Harris
298
2020-02-02
Daniel Gafford
299
2020-02-02
J.J. Redick
300
2020-02-02
Jimmy Butler
301
2020-02-03
Bradley Beal
302
2020-02-03
Kristaps Porziņģis
303
2020-02-03
Kevin Love
304
2020-02-03
Domantas Sabonis
305
2020-02-03
Jonas Valančiūnas
306
2020-02-03
Joel Embiid
307
2020-02-03
LaMarcus Aldridge
308
2020-02-03
Goran Dragić
309
2020-02-03
Alec Burks
310
2020-02-03
John Collins
311
2020-02-03
Elfrid Payton
312
2020-02-03
Deandre Ayton
313
2020-02-03
De'Aaron Fox
314
2020-02-03
Bam Adebayo
315
2020-02-03
Glenn Robinson
316
2020-02-03
Jayson Tatum
317
2020-02-03
Collin Sexton
318
2020-02-03
Trae Young
319
2020-02-03
Andre Drummond
320
2020-02-03
Caris LeVert
321
2020-02-03
Gordon Hayward
322
2020-02-03
Bogdan Bogdanović
323
2020-02-03
Tim Hardaway
324
2020-02-03
Dāvis Bertāns
325
2020-02-03
Cody Martin
326
2020-02-03
Kawhi Leonard
327
2020-02-03
Paul George
328
2020-02-03
Marcus Morris
329
2

Robin Lopez
579
2020-02-04
PJ Dozier
580
2020-02-04
Derrick Favors
581
2020-02-04
Jared Dudley
582
2020-02-04
Keldon Johnson
583
2020-02-04
Trey Lyles
584
2020-02-04
Cody Zeller
585
2020-02-04
Sterling Brown
586
2020-02-04
Rudy Gay
587
2020-02-04
Ersan İlyasova
588
2020-02-04
Patty Mills
589
2020-02-04
Chimezie Metu
590
2020-02-04
Carmelo Anthony
591
2020-02-04
Quinn Cook
592
2020-02-04
Nikola Jokić
593
2020-02-05
Andre Drummond
594
2020-02-05
Trae Young
595
2020-02-05
Kyle Lowry
596
2020-02-05
Kristaps Porziņģis
597
2020-02-05
Jayson Tatum
598
2020-02-05
Derrick Jones
599
2020-02-05
Dennis Schröder
600
2020-02-05
Deandre Ayton
601
2020-02-05
Gordon Hayward
602
2020-02-05
Kelly Oubre
603
2020-02-05
Justin Holiday
604
2020-02-05
Bam Adebayo
605
2020-02-05
Pascal Siakam
606
2020-02-05
Paul George
607
2020-02-05
Serge Ibaka
608
2020-02-05
John Collins
609
2020-02-05
Evan Fournier
610
2020-02-05
Josh Okogie
611
2020-02-05
Shai Gilgeous-Alexander
612
2020-02-05
Reggie Jackson
613
2020-02-05

Frank Ntilikina
875
2020-02-06
Nickeil Alexander-Walker
876
2020-02-06
Kevin Knox
877
2020-02-06
Rajon Rondo
878
2020-02-06
Kelly Oubre
879
2020-02-07
Furkan Korkmaz
880
2020-02-07
Damian Lillard
881
2020-02-07
Devin Booker
882
2020-02-07
Christian Wood
883
2020-02-07
Jayson Tatum
884
2020-02-07
John Collins
885
2020-02-07
Bojan Bogdanović
886
2020-02-07
James Harden
887
2020-02-07
Chris Paul
888
2020-02-07
Kevin Huerter
889
2020-02-07
Reggie Jackson
890
2020-02-07
Ben Simmons
891
2020-02-07
Bam Adebayo
892
2020-02-07
Bogdan Bogdanović
893
2020-02-07
Tobias Harris
894
2020-02-07
Tim Hardaway
895
2020-02-07
Kyle Lowry
896
2020-02-07
Fred VanVleet
897
2020-02-07
Mikal Bridges
898
2020-02-07
Dāvis Bertāns
899
2020-02-07
Jonas Valančiūnas
900
2020-02-07
Mike Conley
901
2020-02-07
Terence Davis
902
2020-02-07
Bradley Beal
903
2020-02-07
Enes Kanter
904
2020-02-07
Danilo Gallinari
905
2020-02-07
T.J. McConnell
906
2020-02-07
Thon Maker
907
2020-02-07
Shai Gilgeous-Alexander
908
2020-02-07
Ga

Thon Maker
1161
2020-02-08
Aaron Gordon
1162
2020-02-08
Courtney Lee
1163
2020-02-08
Terance Mann
1164
2020-02-08
Mfiondu Kabengele
1165
2020-02-08
Jaylen Nowell
1166
2020-02-08
Kyle Korver
1167
2020-02-08
Jalen McDaniels
1168
2020-02-08
Ersan İlyasova
1169
2020-02-08
Cody Zeller
1170
2020-02-08
Jarrett Allen
1171
2020-02-08
Delon Wright
1172
2020-02-08
Keita Bates-Diop
1173
2020-02-08
P.J. Washington
1174
2020-02-08
Allen Crabbe
1175
2020-02-08
Cory Joseph
1176
2020-02-08
E'Twaun Moore
1177
2020-02-08
Donte DiVincenzo
1178
2020-02-08
Kevin Knox
1179
2020-02-08
Patrick McCaw
1180
2020-02-08
Doug McDermott
1181
2020-02-08
Frank Ntilikina
1182
2020-02-08
Juan Toscano-Anderson
1183
2020-02-08
Lonnie Walker
1184
2020-02-08
Amir Coffey
1185
2020-02-08
Rudy Gay
1186
2020-02-08
Rodney McGruder
1187
2020-02-08
Wilson Chandler
1188
2020-02-08
Pat Connaughton
1189
2020-02-08
Yogi Ferrell
1190
2020-02-08
Jordan Bone
1191
2020-02-08
JaMychal Green
1192
2020-02-08
Justin Holiday
1193
2020-02-08
Jev

Myles Turner
1439
2020-02-10
Langston Galloway
1440
2020-02-10
J.J. Barea
1441
2020-02-10
Trey Lyles
1442
2020-02-10
Jarrett Culver
1443
2020-02-10
Thon Maker
1444
2020-02-10
Duncan Robinson
1445
2020-02-10
Sterling Brown
1446
2020-02-10
Jevon Carter
1447
2020-02-10
Juan Hernangómez
1448
2020-02-10
Doug McDermott
1449
2020-02-10
Emmanuel Mudiay
1450
2020-02-10
Rudy Gay
1451
2020-02-10
Draymond Green
1452
2020-02-10
Derrick White
1453
2020-02-10
Mohamed Bamba
1454
2020-02-10
Joe Ingles
1455
2020-02-10
Andrew Wiggins
1456
2020-02-10
Jordan McRae
1457
2020-02-10
Pat Connaughton
1458
2020-02-10
Timothé Luwawu-Cabarrot
1459
2020-02-10
Jalen Brunson
1460
2020-02-10
Wesley Iwundu
1461
2020-02-10
Bogdan Bogdanović
1462
2020-02-10
Donovan Mitchell
1463
2020-02-10
Goran Dragić
1464
2020-02-10
John Henson
1465
2020-02-10
De'Andre Hunter
1466
2020-02-10
Patrick McCaw
1467
2020-02-10
LeBron James
1468
2020-02-10
Cameron Johnson
1469
2020-02-10
P.J. Washington
1470
2020-02-10
Avery Bradley
1471
2020

,name,team,date,location,opponent,minutes,made_two_point_field_goals,made_three_point_field_goals,made_free_throws,rebounds,assists,blocks,steals,turnovers,recent_average
0,Damian Lillard,PORTLAND TRAIL BLAZERS,2020-02-01,HOME,UTAH JAZZ,40.4775,3.7625,3.64002,6.76118,6.24913,7.45448,0.13088,1.13913,0.13088,72.678
1,Bradley Beal,WASHINGTON WIZARDS,2020-02-01,HOME,BROOKLYN NETS,35.819,5.08793,2.66204,8.25835,5.5326,5.792,0.61446,0.83457,0.61446,55.61
2,Karl-Anthony Towns,MINNESOTA TIMBERWOLVES,2020-02-01,AWAY,LOS ANGELES CLIPPERS,34.684,5.82127,2.20194,4.87272,7.63307,4.005,1.11784,0.90033,1.11784,49.427
3,Dorian Finney-Smith,DALLAS MAVERICKS,2020-02-01,HOME,ATLANTA HAWKS,29.3773,2.34479,1.4722,1.70572,7.45585,2.6266,0.89208,0.81144,0.89208,19.787
4,Anthony Davis,LOS ANGELES LAKERS,2020-02-01,AWAY,SACRAMENTO KINGS,33.169,6.09423,1.5257,5.23454,7.3915,3.84436,1.36416,1.08783,1.36416,51.917
5,Kawhi Leonard,LOS ANGELES CLIPPERS,2020-02-01,HOME,MINNESOTA TIMBERWOLVES,35.1078,6.29492,2.2854,5.9465,7.42046,5.37268,0.78188,1.42473,0.78188,62.897
6,Jalen Brunson,DALLAS MAVERICKS,2020-02-01,HOME,ATLANTA HAWKS,19.1787,4.3244,1.04206,1.98574,4.41373,3.27504,0.46386,0.82161,0.46386,16.586
7,Jaylen Brown,BOSTON CELTICS,2020-02-01,HOME,PHILADELPHIA 76ERS,33.9642,3.91225,2.06926,3.23376,5.94979,3.6906,0.67306,1.03386,0.67306,32.815
8,DeMar DeRozan,SAN ANTONIO SPURS,2020-02-01,HOME,CHARLOTTE HORNETS,36.1353,8.0507,0.83876,8.50702,5.89453,4.33164,0.21466,0.83382,0.21466,42.981
9,Cody Zeller,CHARLOTTE HORNETS,2020-02-01,AWAY,SAN ANTONIO SPURS,24.0365,5.16858,0.97144,2.54816,7.13337,3.25344,1.10474,0.8835,1.10474,23.906


In [22]:
import pytz

def predict_next_day(start_date, game_date):
    season_schedule = client.season_schedule(season_end_year = game_date.year)
    schedule_on_date = [game for game in season_schedule if (game['start_time'] - timedelta(hours = 4)).day == game_date.day and game['start_time'].month == game_date.month]
    team_objs_on_date = [game["home_team"] for game in schedule_on_date] + [game["away_team"] for game in schedule_on_date]
    teams_on_date = [t.name.replace("_", " ") for t in team_objs_on_date]
    predictions_for_date = make_predictions(start_date, game_date, output = False)
    players_on_date = predictions_for_date[predictions_for_date.team.isin(teams_on_date)]
    mstr, dstr = str(game_date.month), str(game_date.day)
    if game_date.month < 10:
        mstr = "0" + mstr
    if game_date.day < 10:
        dstr = "0" + dstr
    tstr = mstr + "-" + dstr
    statlines_on_date = players_on_date[players_on_date.date.str.contains(tstr)]
    statlines_on_date.index = range(statlines_on_date.shape[0])
    output_filename = './AllCSVs/predictions_for_' + mstr + "_" + dstr + '_from_' + str(start_date.month) + '_' + str(start_date.day) + '_2020.csv'
    statlines_on_date.to_csv(output_filename)
    return statlines_on_date
predict_next_day(datetime(2020, 2, 1), datetime(2020, 2, 12))

Damian Lillard
0
2020-02-01
Bradley Beal
1
2020-02-01
Karl-Anthony Towns
2
2020-02-01
Dorian Finney-Smith
3
2020-02-01
Anthony Davis
4
2020-02-01
Kawhi Leonard
5
2020-02-01
Jalen Brunson
6
2020-02-01
Jaylen Brown
7
2020-02-01
DeMar DeRozan
8
2020-02-01
Cody Zeller
9
2020-02-01
Glenn Robinson
10
2020-02-01
Hassan Whiteside
11
2020-02-01
Collin Sexton
12
2020-02-01
Maxi Kleber
13
2020-02-01
Draymond Green
14
2020-02-01
Nikola Vučević
15
2020-02-01
Jayson Tatum
16
2020-02-01
Jarrett Allen
17
2020-02-01
John Collins
18
2020-02-01
Ben Simmons
19
2020-02-01
Tyler Herro
20
2020-02-01
Jakob Poeltl
21
2020-02-01
Aaron Gordon
22
2020-02-01
Bojan Bogdanović
23
2020-02-01
Miles Bridges
24
2020-02-01
Marcus Morris
25
2020-02-01
D'Angelo Russell
26
2020-02-01
Jimmy Butler
27
2020-02-01
Domantas Sabonis
28
2020-02-01
Paul George
29
2020-02-01
Buddy Hield
30
2020-02-01
Donovan Mitchell
31
2020-02-01
Harry Giles
32
2020-02-01
LeBron James
33
2020-02-01
Damion Lee
34
2020-02-01
Joe Harris
35
2020-02-01


Gary Harris
298
2020-02-02
Daniel Gafford
299
2020-02-02
J.J. Redick
300
2020-02-02
Jimmy Butler
301
2020-02-03
Bradley Beal
302
2020-02-03
Kristaps Porziņģis
303
2020-02-03
Kevin Love
304
2020-02-03
Domantas Sabonis
305
2020-02-03
Jonas Valančiūnas
306
2020-02-03
Joel Embiid
307
2020-02-03
LaMarcus Aldridge
308
2020-02-03
Goran Dragić
309
2020-02-03
Alec Burks
310
2020-02-03
John Collins
311
2020-02-03
Elfrid Payton
312
2020-02-03
Deandre Ayton
313
2020-02-03
De'Aaron Fox
314
2020-02-03
Bam Adebayo
315
2020-02-03
Glenn Robinson
316
2020-02-03
Jayson Tatum
317
2020-02-03
Collin Sexton
318
2020-02-03
Trae Young
319
2020-02-03
Andre Drummond
320
2020-02-03
Caris LeVert
321
2020-02-03
Gordon Hayward
322
2020-02-03
Bogdan Bogdanović
323
2020-02-03
Tim Hardaway
324
2020-02-03
Dāvis Bertāns
325
2020-02-03
Cody Martin
326
2020-02-03
Kawhi Leonard
327
2020-02-03
Paul George
328
2020-02-03
Marcus Morris
329
2020-02-03
Markelle Fultz
330
2020-02-03
Karl-Anthony Towns
331
2020-02-03
Nikola Vučevi

Rudy Gay
587
2020-02-04
Ersan İlyasova
588
2020-02-04
Patty Mills
589
2020-02-04
Chimezie Metu
590
2020-02-04
Carmelo Anthony
591
2020-02-04
Quinn Cook
592
2020-02-04
Nikola Jokić
593
2020-02-05
Andre Drummond
594
2020-02-05
Trae Young
595
2020-02-05
Kyle Lowry
596
2020-02-05
Kristaps Porziņģis
597
2020-02-05
Jayson Tatum
598
2020-02-05
Derrick Jones
599
2020-02-05
Dennis Schröder
600
2020-02-05
Deandre Ayton
601
2020-02-05
Gordon Hayward
602
2020-02-05
Kelly Oubre
603
2020-02-05
Justin Holiday
604
2020-02-05
Bam Adebayo
605
2020-02-05
Pascal Siakam
606
2020-02-05
Paul George
607
2020-02-05
Serge Ibaka
608
2020-02-05
John Collins
609
2020-02-05
Evan Fournier
610
2020-02-05
Josh Okogie
611
2020-02-05
Shai Gilgeous-Alexander
612
2020-02-05
Reggie Jackson
613
2020-02-05
Tyus Jones
614
2020-02-05
Christian Wood
615
2020-02-05
Malcolm Brogdon
616
2020-02-05
Jamal Murray
617
2020-02-05
Brandon Clarke
618
2020-02-05
Mike Conley
619
2020-02-05
Aaron Gordon
620
2020-02-05
Landry Shamet
621
2020

Rajon Rondo
878
2020-02-06
Kelly Oubre
879
2020-02-07
Furkan Korkmaz
880
2020-02-07
Damian Lillard
881
2020-02-07
Devin Booker
882
2020-02-07
Christian Wood
883
2020-02-07
Jayson Tatum
884
2020-02-07
John Collins
885
2020-02-07
Bojan Bogdanović
886
2020-02-07
James Harden
887
2020-02-07
Chris Paul
888
2020-02-07
Kevin Huerter
889
2020-02-07
Reggie Jackson
890
2020-02-07
Ben Simmons
891
2020-02-07
Bam Adebayo
892
2020-02-07
Bogdan Bogdanović
893
2020-02-07
Tobias Harris
894
2020-02-07
Tim Hardaway
895
2020-02-07
Kyle Lowry
896
2020-02-07
Fred VanVleet
897
2020-02-07
Mikal Bridges
898
2020-02-07
Dāvis Bertāns
899
2020-02-07
Jonas Valančiūnas
900
2020-02-07
Mike Conley
901
2020-02-07
Terence Davis
902
2020-02-07
Bradley Beal
903
2020-02-07
Enes Kanter
904
2020-02-07
Danilo Gallinari
905
2020-02-07
T.J. McConnell
906
2020-02-07
Thon Maker
907
2020-02-07
Shai Gilgeous-Alexander
908
2020-02-07
Gary Trent
909
2020-02-07
CJ McCollum
910
2020-02-07
Rudy Gobert
911
2020-02-07
OG Anunoby
912
2020

Aaron Gordon
1162
2020-02-08
Courtney Lee
1163
2020-02-08
Terance Mann
1164
2020-02-08
Mfiondu Kabengele
1165
2020-02-08
Jaylen Nowell
1166
2020-02-08
Kyle Korver
1167
2020-02-08
Jalen McDaniels
1168
2020-02-08
Ersan İlyasova
1169
2020-02-08
Cody Zeller
1170
2020-02-08
Jarrett Allen
1171
2020-02-08
Delon Wright
1172
2020-02-08
Keita Bates-Diop
1173
2020-02-08
P.J. Washington
1174
2020-02-08
Allen Crabbe
1175
2020-02-08
Cory Joseph
1176
2020-02-08
E'Twaun Moore
1177
2020-02-08
Donte DiVincenzo
1178
2020-02-08
Kevin Knox
1179
2020-02-08
Patrick McCaw
1180
2020-02-08
Doug McDermott
1181
2020-02-08
Frank Ntilikina
1182
2020-02-08
Juan Toscano-Anderson
1183
2020-02-08
Lonnie Walker
1184
2020-02-08
Amir Coffey
1185
2020-02-08
Rudy Gay
1186
2020-02-08
Rodney McGruder
1187
2020-02-08
Wilson Chandler
1188
2020-02-08
Pat Connaughton
1189
2020-02-08
Yogi Ferrell
1190
2020-02-08
Jordan Bone
1191
2020-02-08
JaMychal Green
1192
2020-02-08
Justin Holiday
1193
2020-02-08
Jevon Carter
1194
2020-02-08
T

Sterling Brown
1446
2020-02-10
Jevon Carter
1447
2020-02-10
Juan Hernangómez
1448
2020-02-10
Doug McDermott
1449
2020-02-10
Emmanuel Mudiay
1450
2020-02-10
Rudy Gay
1451
2020-02-10
Draymond Green
1452
2020-02-10
Derrick White
1453
2020-02-10
Mohamed Bamba
1454
2020-02-10
Joe Ingles
1455
2020-02-10
Andrew Wiggins
1456
2020-02-10
Jordan McRae
1457
2020-02-10
Pat Connaughton
1458
2020-02-10
Timothé Luwawu-Cabarrot
1459
2020-02-10
Jalen Brunson
1460
2020-02-10
Wesley Iwundu
1461
2020-02-10
Bogdan Bogdanović
1462
2020-02-10
Donovan Mitchell
1463
2020-02-10
Goran Dragić
1464
2020-02-10
John Henson
1465
2020-02-10
De'Andre Hunter
1466
2020-02-10
Patrick McCaw
1467
2020-02-10
LeBron James
1468
2020-02-10
Cameron Johnson
1469
2020-02-10
P.J. Washington
1470
2020-02-10
Avery Bradley
1471
2020-02-10
Jordan Poole
1472
2020-02-10
Cory Joseph
1473
2020-02-10
Garrett Temple
1474
2020-02-10
Victor Oladipo
1475
2020-02-10
Patty Mills
1476
2020-02-10
Donte DiVincenzo
1477
2020-02-10
Kevin Huerter
1478
2

Sterling Brown
1730
2020-02-12
Jalen McDaniels
1731
2020-02-12
Miles Bridges
1732
2020-02-12
Troy Brown
1733
2020-02-12
Monte Morris
1734
2020-02-12
Jae Crowder
1735
2020-02-12
Paul Millsap
1736
2020-02-12
Duncan Robinson
1737
2020-02-12
Cheick Diallo
1738
2020-02-12
Andre Drummond
1739
2020-02-12
Kyle Lowry
1740
2020-02-12
Pascal Siakam
1741
2020-02-12
Kent Bazemore
1742
2020-02-12
Alex Caruso
1743
2020-02-12
Cory Joseph
1744
2020-02-12
Mitchell Robinson
1745
2020-02-12
Trevor Ariza
1746
2020-02-12
Juan Hernangómez
1747
2020-02-12
Mikal Bridges
1748
2020-02-12
Fred VanVleet
1749
2020-02-12
Naz Reid
1750
2020-02-12
Brook Lopez
1751
2020-02-12
Jevon Carter
1752
2020-02-12
Dwight Howard
1753
2020-02-12
Cam Reddish
1754
2020-02-12
Harry Giles
1755
2020-02-12
Victor Oladipo
1756
2020-02-12
DeAndre Jordan
1757
2020-02-12
Tony Bradley
1758
2020-02-12
De'Aaron Fox
1759
2020-02-12
Josh Okogie
1760
2020-02-12
Dewayne Dedmon
1761
2020-02-12
Wesley Iwundu
1762
2020-02-12
Damion Lee
1763
2020-02-1

,name,team,date,location,opponent,minutes,made_two_point_field_goals,made_three_point_field_goals,made_free_throws,rebounds,assists,blocks,steals,turnovers,recent_average
0,T.J. Warren,INDIANA PACERS,2020-02-12,HOME,MILWAUKEE BUCKS,30.8353,5.56276,1.13408,3.64134,4.3516,2.68364,0.5115,1.20984,0.5115,27.633
1,Aaron Gordon,ORLANDO MAGIC,2020-02-12,HOME,DETROIT PISTONS,34.4602,5.07748,1.73542,3.44812,7.71761,4.4518,0.77934,1.01517,0.77934,37.629
2,Andrew Wiggins,GOLDEN STATE WARRIORS,2020-02-12,AWAY,PHOENIX SUNS,32.9885,4.04974,1.91732,4.95377,5.1045,3.93808,0.58656,1.33827,0.58656,32.614
3,Luka Dončić,DALLAS MAVERICKS,2020-02-12,HOME,SACRAMENTO KINGS,31.9093,4.63657,2.71126,4.7777,6.47619,5.0132,0.39632,1.05129,0.39632,44.766
4,Kristaps Porziņģis,DALLAS MAVERICKS,2020-02-12,HOME,SACRAMENTO KINGS,30.9818,2.89216,2.62138,6.37248,7.06805,2.5982,0.77462,1.23033,0.77462,46.337
5,Tristan Thompson,CLEVELAND CAVALIERS,2020-02-12,HOME,ATLANTA HAWKS,23.7022,4.25839,0.93506,1.66283,8.43243,2.90648,0.81716,0.7656,0.81716,20.38
6,Brandon Clarke,MEMPHIS GRIZZLIES,2020-02-12,HOME,PORTLAND TRAIL BLAZERS,22.9283,4.9911,0.99498,2.23169,6.34291,2.52532,0.73782,0.93801,0.73782,21.643
7,LeBron James,LOS ANGELES LAKERS,2020-02-12,AWAY,DENVER NUGGETS,33.2082,4.35571,2.27898,3.55023,6.22527,7.83752,0.13096,1.11672,0.13096,45.436
8,Larry Nance,CLEVELAND CAVALIERS,2020-02-12,HOME,ATLANTA HAWKS,28.2375,3.29944,1.64982,0.834482,6.62359,3.44464,0.57254,0.73851,0.57254,23.34
9,Anthony Davis,LOS ANGELES LAKERS,2020-02-12,AWAY,DENVER NUGGETS,33.289,7.15983,1.04206,5.83771,7.46696,4.05076,0.9372,1.34385,0.9372,49.583


In [33]:
def append_actual_results(start_date, game_date):
    predicted_results = predict_next_day(start_date, game_date)
    actual_results = box_scores_for_range_of_days(game_date, game_date)
    actual_results["made_two_point_field_goals"] = actual_results["made_field_goals"] - actual_results["made_three_point_field_goals"]
    actual_results["rebounds"] = actual_results["offensive_rebounds"] + actual_results["defensive_rebounds"]
    pred_outputs = [float(get_points(predicted_results[predicted_results["name"] == player_name])[0]) for player_name in predicted_results["name"]]
    actual_outputs = [float(get_points(actual_results[actual_results["name"] == player_name])[0]) for player_name in predicted_results["name"]]
    predicted_results["predicted_points"] = pred_outputs
    predicted_results["actual_points"] = actual_outputs
    output_filename = './AllCSVs/' + str(game_date.month) + '_' + str(game_date.day) + '_2020_box_scores_predicted_with_actual.csv'
    predicted_results.to_csv(output_filename)
    return predicted_results

append_actual_results(datetime(2020, 2, 1), datetime(2020, 2, 12))

Damian Lillard
0
2020-02-01
Dāvis Bertāns
50
2020-02-01
Willy Hernangómez
100
2020-02-01
Marvin Williams
150
2020-02-01
Wenyen Gabriel
200
2020-02-01
Fred VanVleet
250
2020-02-02
J.J. Redick
300
2020-02-02
Shabazz Napier
350
2020-02-03
Delon Wright
400
2020-02-03
Gary Clark
450
2020-02-03
Matthew Dellavedova
500
2020-02-03
Cody Martin
550
2020-02-04
Dennis Schröder
600
2020-02-05
Bojan Bogdanović
650
2020-02-05
DeAndre Jordan
700
2020-02-05
Chris Silva
750
2020-02-05
Danny Green
800
2020-02-06
JaVale McGee
850
2020-02-06
Jonas Valančiūnas
900
2020-02-07
Nerlens Noel
950
2020-02-07
Goga Bitadze
1000
2020-02-07
Jordan McLaughlin
1050
2020-02-08
Miles Bridges
1100
2020-02-08
Damion Lee
1150
2020-02-08
Wesley Iwundu
1200
2020-02-08
Shai Gilgeous-Alexander
1250
2020-02-09
Derrick Jones
1300
2020-02-09
Abdel Nader
1350
2020-02-09
Spencer Dinwiddie
1400
2020-02-10
Emmanuel Mudiay
1450
2020-02-10
James Johnson
1500
2020-02-10
Alex Caruso
1550
2020-02-10
Steven Adams
1600
2020-02-11
Carsen Edwa

,name,team,date,location,opponent,minutes,made_two_point_field_goals,made_three_point_field_goals,made_free_throws,rebounds,assists,blocks,steals,turnovers,recent_average,predicted_points,actual_points
0,T.J. Warren,INDIANA PACERS,2020-02-12,HOME,MILWAUKEE BUCKS,30.8353,5.56276,1.13408,3.64134,4.3516,2.68364,0.5115,1.20984,0.5115,27.633,32.068992,54.9
1,Aaron Gordon,ORLANDO MAGIC,2020-02-12,HOME,DETROIT PISTONS,34.4602,5.07748,1.73542,3.44812,7.71761,4.4518,0.77934,1.01517,0.77934,37.629,39.352364,54.3
2,Andrew Wiggins,GOLDEN STATE WARRIORS,2020-02-12,AWAY,PHOENIX SUNS,32.9885,4.04974,1.91732,4.95377,5.1045,3.93808,0.58656,1.33827,0.58656,32.614,36.025657,55.3
3,Luka Dončić,DALLAS MAVERICKS,2020-02-12,HOME,SACRAMENTO KINGS,31.9093,4.63657,2.71126,4.7777,6.47619,5.0132,0.39632,1.05129,0.39632,44.766,41.422347,55.4
4,Kristaps Porziņģis,DALLAS MAVERICKS,2020-02-12,HOME,SACRAMENTO KINGS,30.9818,2.89216,2.62138,6.37248,7.06805,2.5982,0.77462,1.23033,0.77462,46.337,37.640130,58.1
5,Tristan Thompson,CLEVELAND CAVALIERS,2020-02-12,HOME,ATLANTA HAWKS,23.7022,4.25839,0.93506,1.66283,8.43243,2.90648,0.81716,0.7656,0.81716,20.38,31.394545,44.7
6,Brandon Clarke,MEMPHIS GRIZZLIES,2020-02-12,HOME,PORTLAND TRAIL BLAZERS,22.9283,4.9911,0.99498,2.23169,6.34291,2.52532,0.73782,0.93801,0.73782,21.643,30.887962,44.7
7,LeBron James,LOS ANGELES LAKERS,2020-02-12,AWAY,DENVER NUGGETS,33.2082,4.35571,2.27898,3.55023,6.22527,7.83752,0.13096,1.11672,0.13096,45.436,41.937272,64.4
8,Larry Nance,CLEVELAND CAVALIERS,2020-02-12,HOME,ATLANTA HAWKS,28.2375,3.29944,1.64982,0.834482,6.62359,3.44464,0.57254,0.73851,0.57254,23.34,28.858710,52.4
9,Anthony Davis,LOS ANGELES LAKERS,2020-02-12,AWAY,DENVER NUGGETS,33.289,7.15983,1.04206,5.83771,7.46696,4.05076,0.9372,1.34385,0.9372,49.583,44.225982,58.0


In [ ]:
def alternate_contests(start_date, game_date, search_datetime, inclusive):
    season_schedule = client.season_schedule(season_end_year = game_date.year)
    schedule_on_date = [game for game in season_schedule if (game['start_time'] - timedelta(hours = 4)).day == game_date.day and game['start_time'].month == game_date.month]
    times_on_date = np.unique([game['start_time'] - timedelta(hours = 4) for game in schedule_on_date])
    print(times_on_date)
    if not inclusive:
        game_teams = [game["home_team"] for game in schedule_on_date if (game['start_time'] - timedelta(hours = 4)) == search_datetime] + [game["away_team"] for game in schedule_on_date if (game['start_time'] - timedelta(hours = 4)) == search_datetime]
    else:
        times_after_search = [t for t in times_on_date if 60 * t.hour + t.minute >= 60 * search_datetime.hour + search_datetime.minute]
        game_teams = [game["home_team"] for game in schedule_on_date if (game['start_time'] - timedelta(hours = 4)) in times_after_search] + [game["away_team"] for game in schedule_on_date if (game['start_time'] - timedelta(hours = 4)) in times_after_search]
    print([team["Team"] for team in game_teams])
    predictions_for_date = predict_next_day(start_date, game_date)
    players_on_date = predictions_for_date[predictions_for_date.team.isin(game_teams)]
    output_filename = './AllCSVs/' + str(game_date.month) + '_' + str(game_date.day) + '_2020_alternate_' + str(search_datetime.hour) + "_" + str(search_datetime.minute) + '.csv'
    players_on_date.to_csv(output_filename)
    return players_on_date
alternate_contests(datetime(2020, 1, 1), datetime(2020, 1, 13), datetime(2020, 1, 13, 21, 30), True)

In [ ]:
def optimal_lineup_games(input_games, ):

In [30]:
all_abbrv = {'ATL':'Atlanta', 'BOS':'Boston', 'BKN':'Brooklyn', 'CHA':'Charlotte', 'CHI':'Chicago', 'CLE':'Cleveland', 'DAL':'Dallas',
            'DEN':'Denver', 'DET':'Detroit', 'GSW':'Golden State', 'HOU':'Houston', 'IND':'Indiana', 'LAC':'LA Clippers', 'LAL':'LA Lakers',
            'MEM':'Memphis', 'MIA':'Miami', 'MIL':'Milwaukee', 'MIN':'Minnesota', 'NOR':'New Orleans', 'NYK':'New York', 'OKC':'Oklahoma City', 'ORL':'Orlando',
            'PHI':'Philadelphia', 'PHO':'Phoenix', 'POR':'Portland', 'SAC':'Sacramento', 'SAS':'San Antonio', 'TOR':'Toronto', 'UTA':'Utah', 'WAS':'Washington'}
dkp = pd.read_csv('./OutputCSVs/draftkings_points.csv')
dkp = dkp[pd.isna(dkp.position)==False]
dbp = pd.read_csv('./OutputCSVs/team_def_vs_position.csv')

ranks = []
seen = {}
player_names = [seen.setdefault(x, x) for x in dkp['name'] if x not in seen]
abbrv = [str(dkp.loc[dkp['name']==x, 'team'].iloc[0]).upper() for x in player_names]
player_positions = [dkp.loc[dkp['name']==x, 'position'].iloc[0] for x in player_names]
team_names = [all_abbrv.get(x) for x in abbrv]

for i in range(0, len(player_names)):
    if len(player_positions[i])<=2:
        ranks.append(dbp.loc[dbp['Team']==team_names[i], 'vs {0}'.format(player_positions[i])].iloc[0])
    else:
        first_subrank =  dbp.loc[dbp['Team']==team_names[i], 'vs {0}'.format(player_positions[i][:2])].iloc[0]
        second_subrank =  dbp.loc[dbp['Team']==team_names[i], 'vs {0}'.format(player_positions[i][3:])].iloc[0]
        ranks.append((first_subrank + second_subrank)/2)

outputDefense = pd.DataFrame({'name':player_names, 'pos':player_positions, 'team':team_names, 'rank':ranks})
outputDefense.to_csv('./OutputCSVs/player_def_vs_position.csv', header=True, index=False)